# Используем f2py

Я очень советую воспользоваться [этим гайдом](https://numpy.org/devdocs/f2py/f2py.getting-started.html), он простой и довольно понятный. Давайте посмотрим на пример оттуда:

In [7]:
cat fib1.f

C FILE: FIB1.F
      SUBROUTINE FIB(A,N)
C
C     CALCULATE FIRST N FIBONACCI NUMBERS
C
      INTEGER N
      REAL*8 A(N)
      DO I=1,N
         IF (I.EQ.1) THEN
            A(I) = 0.0D0
         ELSEIF (I.EQ.2) THEN
            A(I) = 1.0D0
         ELSE 
            A(I) = A(I-1) + A(I-2)
         ENDIF
      ENDDO
      END
C END FILE FIB1.F


Теперь надо скомпилировать эту штуку с помощью `f2py`. Если вы используете unix-подобные системы, то все относительно просто, главное не забудьте поставить `gfortran` и `gcc`, дальше вбиваем в командную линию из директории в которой находится этот файл:

`python -m numpy.f2py -c fib1.f -m fib1`

По какой-то причине это выдавало у меня невнятную ошибку, но почему-то заработало от sudo, прада пришлось указать путь к питону, у меня было так:

`sudo /home/teimy/anaconda3/bin/python -m numpy.f2py -c fib1.f -m fib1`

Не знаю, как будет работать у вас, попробуйте оба варианта, главное не забудьте поменять путь к питону так, как он прописан у вас. Если вы не знаете где он, то можно это узнать прописав `which python`.

С windows история чуть более сложная, опять же есть [вот такой гайд](https://scipy.github.io/old-wiki/pages/F2PY_Windows.html), а если он не поможет, то [этот ответ на стаковерфлоу](https://stackoverflow.com/questions/48826283/compile-fortran-module-with-f2py-and-python-3-6-on-windows-10). 

Теперь у нас есть файл `fib1.so` (или что-то в этом духе, у меня вот `fib1.cpython-37m-x86_64-linux-gnu.so`), который является своеобразным модулем, в котором находятся все функции из файла `fib1.f`. Посмотрим как он работает:

In [9]:
import fib1
import numpy as np

In [33]:
a = np.zeros(8)
fib1.fib(a, 8)

In [34]:
a

array([ 0.,  1.,  1.,  2.,  3.,  5.,  8., 13.])

В целом оно работает! Но немного непривычно, я как-то привык что функция что-то возвращает. На самом деле такая схема ~~почти никогда не~~ не всегда работает, поэтому надо размечать аргументы функции в самом файле. Вот так:

In [36]:
cat fib3.f

C FILE: FIB3.F
      SUBROUTINE FIB(A,N)
C
C     CALCULATE FIRST N FIBONACCI NUMBERS
C
      INTEGER N
      REAL*8 A(N)
Cf2py intent(in) n
Cf2py intent(out) a
Cf2py depend(n) a
      DO I=1,N
         IF (I.EQ.1) THEN
            A(I) = 0.0D0
         ELSEIF (I.EQ.2) THEN
            A(I) = 1.0D0
         ELSE 
            A(I) = A(I-1) + A(I-2)
         ENDIF
      ENDDO
      END
C END FILE FIB3.F

Так как первый символ строки C, то fortran будет считать эту строку комменатарием, и будет ее игнорировать, а вот cython ее читает и запоминает. Посмотрим:

In [41]:
import fib3
fib3.fib(8)

array([ 0.,  1.,  1.,  2.,  3.,  5.,  8., 13.])

Перейдем к более серьезным примерам. Возьмем целый набор функций `spher.f`:

In [46]:
cat spher_test.f


C  The program computes far-field light scattering by polydisperse     
C  homogeneous spherical particles using the Lorenz-Mie theory.    
                                                                     
C  Last modified 08/06/2005                                            
                                                                        
C  The program is based on the book                                    
C 
C  1. Mishchenko, M. I., L. D. Travis, and A. A. Lacis (2002). 
C     Scattering, Absorption, and Emission of Light
C     by Small Particles. Cambridge University Press, Cambridge.
C
C  The book is available in the .pdf format at the following
C  web site:
C
C   http://www.giss.nasa.gov/~crmim/books.html
C                                                                      
C  One can also use the following paper as a brief user guide:               
C                                                                      
C  2. M. I. Mishchenko, J. M. Dlugach, E.

In [47]:
import spher_test

Пропишем какие-нибудь дефолотные аргументы, чтобы убедиться в работспособности:

In [48]:
r1 = 0.1
r2 = 10
ndistr = 2 # log-normal disribution
aa = 1 # mean radius of distribution, let us say in microns
bb = np.log(0.7)**2 # geom_std here is geometrical std. dev
gam = 1 # most likely you do not need in case of log distr this one but who knows?
lam = 1.3 # lambda in microns
mrr = 1.5 # real part of refr. index
mrii = 0.01 # im. part of refr. index
n = 100 # number of integration subintervals on the interval (R1, R2)  of particle radii 
        # 100 was in example, and it works good
        # note that r1 and r2 seriously affect this parameter
npp = 1000 # for power law  
nk = 5 # number of gaussian division points. looks legit at 3.
# npna = 10 # also dont know what is good value
# next parameters should be useless:
area = 0 # ??? 
volume = 0 # ????
rvw = 0 # ??
rmean = 0 # ???
bb2 = 0 #????
bb1 = 0 #???
aa1 = 0 #???
aa2 = 0 #???
ddelt = 1e-14 # I do not know how exactly it affects computations.

L1 = 0
AL1 = 0 
AL2 = 0
AL3 = 0
AL4 = 0
BET1 = 0
BET2 = 0
CEXT = 0
CSCA = 0

In [49]:
l1,al1,al2,al3,al4,bet1,bet2,cext,csca = spher_test.spher(aa,bb,gam,lam,mrr,mrii,r1,r2,n,npp,ndistr,nk,area,volume,rvw,rmean,aa1,bb1,aa2,bb2,ddelt)

In [50]:
csca/cext

0.8947042815350186

Если вы не хотите таскать `.so` файл из проекта в проект, то можете его закинуть `/home/user/anaconda3/bin`, ну или туда где у вас стоит anaconda.